# Pet adoptions with deep networks
This simple project aims to build a simple deep neural network with tensorflow with automated hyperparameter tuning by predicting adoptions of animals. The project follows https://www.tensorflow.org/tutorials/structured_data/feature_columns, with hyperparameter tuning inspired by https://www.tensorflow.org/tutorials/keras/keras_tuner. Feature comlumns follows tanzhenyu comments on Jun 15, 2019 from https://github.com/tensorflow/tensorflow/issues/27416#issuecomment-502218673, because suche example allows to use feature columns in conjunction with the functional API of keras.

### Library import
As first thing, I import the libraries important for this project

In [1]:
# import complete libraries
import numpy
import pandas
import tensorflow
import kerastuner
import os

# import sub-libraries and specific functions
from tensorflow import feature_column
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from tensorboard.plugins.hparams import api as hp

### Data import and create train, validate, test dataset
Download the dataset with the keras get_file utility, and import it as a pandas dataframe

In [2]:
dataset_url = 'http://storage.googleapis.com/download.tensorflow.org/data/petfinder-mini.zip'
csv_file = 'datasets/petfinder-mini/petfinder-mini.csv'

tensorflow.keras.utils.get_file('petfinder_mini.zip', dataset_url,
                        extract=True, cache_dir='.')
dataframe = pandas.read_csv(csv_file)


Construct labels upon using the information that AdoptionSpeed = 4 labels animals that were not adopted, and drop columns of no interest. 

In [3]:
# Encode data labels
dataframe['target'] = numpy.where(dataframe['AdoptionSpeed']==4, 0, 1)

# Drop un-used columns.
dataframe = dataframe.drop(columns=['AdoptionSpeed', 'Description'])

At this point, a normal project would include data exploration. I skip it here because this prokject has different aims.

Split dataset into train, validation and test datasets.

In [4]:
train, test = train_test_split(dataframe, test_size=0.2, random_state = 0)
train, val = train_test_split(train, test_size=0.2, random_state = 0)
print(len(train), 'train examples')
print(len(val), 'validation examples')
print(len(test), 'test examples')


7383 train examples
1846 validation examples
2308 test examples


Create datasets from dataframe using utilities from the GCP platform

In [5]:
# A utility method to create a tf.data dataset from a Pandas Dataframe
def df_to_dataset(dataframe, shuffle=True, batch_size=32):
  dataframe = dataframe.copy()
  labels = dataframe.pop('target')
  ds = tensorflow.data.Dataset.from_tensor_slices((dict(dataframe), labels))
  if shuffle:
    ds = ds.shuffle(buffer_size=len(dataframe))
  ds = ds.batch(batch_size)
  return ds

batch_size = 32
train_ds = df_to_dataset(train, batch_size=batch_size)
val_ds = df_to_dataset(val, shuffle=False, batch_size=batch_size)
test_ds = df_to_dataset(test, shuffle=False, batch_size=batch_size)


And below, just a few extra utilities that helps with the job of inspecting stuff

In [6]:
# extract one batch to play around
batch, label = iter(train_ds).next()

# Utility to visualize the dataset structure
for key, value in batch.items():
    print(f"{key:20s}: {value}")
print(f"{'label':20s}: {label}")

# utility to inspect the dataset composition
def demo(feature_column):
  feature_layer = layers.DenseFeatures(feature_column)
  print(feature_layer(batch).numpy())


Type                : [b'Dog' b'Dog' b'Cat' b'Dog' b'Cat' b'Cat' b'Cat' b'Cat' b'Cat' b'Dog'
 b'Dog' b'Cat' b'Cat' b'Cat' b'Cat' b'Dog' b'Cat' b'Dog' b'Cat' b'Cat'
 b'Cat' b'Cat' b'Cat' b'Dog' b'Dog' b'Dog' b'Cat' b'Dog' b'Dog' b'Dog'
 b'Dog' b'Cat']
Age                 : [12 84  3  2 24  5  7  1  2  3 12  3  4  3  4 24  2  8 14  6  2  6 24  6
 48 72  1  3  1  2  2  5]
Breed1              : [b'Mixed Breed' b'Shar Pei' b'Domestic Short Hair' b'Mixed Breed'
 b'Domestic Short Hair' b'Domestic Short Hair' b'Domestic Long Hair'
 b'Domestic Short Hair' b'Domestic Short Hair' b'Mixed Breed'
 b'Jack Russell Terrier' b'Domestic Long Hair' b'Domestic Short Hair'
 b'Bengal' b'Domestic Short Hair' b'Beagle' b'Siamese' b'Boston Terrier'
 b'Domestic Long Hair' b'Domestic Short Hair' b'Domestic Short Hair'
 b'Maine Coon' b'Persian' b'Mixed Breed' b'Mixed Breed' b'Mixed Breed'
 b'Tabby' b'Mixed Breed' b'Mixed Breed' b'Mixed Breed'
 b'Black Labrador Retriever' b'Domestic Short Hair']
Gender            

### Build feature columns
Okay, time to build the feature column. First, let's create the groups of basic features that I want to include.

In [7]:
# purely numeric features
numeric_features = ['PhotoAmt', 
                    'Fee']

# bucketized features, with buckets to use in a feature:bucket dictionary form
bucketized_features = {'Age': [1, 2, 3, 4, 5]}

# indicator features
indicator_features = ['Type', 
                      'Color1', 
                      'Color2', 
                      'Gender', 
                      'MaturitySize',
                      'FurLength', 
                      'Vaccinated', 
                      'Sterilized', 
                      'Health']

# embedded features
embedded_features = ['Breed1']



And now, let's define the feature columns and the layer that allows to input the feature columns into a tensorflow neural network model following tanzhenyu comments on Jun 15, 2019 from https://github.com/tensorflow/tensorflow/issues/27416#issuecomment-502218673 to integrate the functional API of keras with feature columns functionalities. Note that you can apply the demo utility on each new_feature separately, or on the overall feature_columns array as a whole.

In [21]:
# function to build the feature columns and the input layer to feed the feature columns in the neural
# network. The original pandas dataframe is referenced as global variable.
# Note also that the input_layer contains only the variables that appear in the original dataframe
def build_feature_columns_and_input_layer():
    feature_columns = []
    input_layer = {}

    # add numeric features to feature columns and input layer
    for feature_name in numeric_features:
        new_feature = feature_column.numeric_column(feature_name)
        feature_columns.append(new_feature)
        input_layer[feature_name] = tensorflow.keras.Input(shape=(1,), name = feature_name)

    # add bucketized features from numeric    
    for feature_name in bucketized_features:
        new_feature = feature_column.bucketized_column(feature_column.numeric_column(feature_name),
                                                       bucketized_features[feature_name])
        feature_columns.append(new_feature)
        input_layer[feature_name] = tensorflow.keras.Input(shape=(1,), name = feature_name)

    # add indicator feature
    for feature_name in indicator_features:
        new_feature_as_categorical = feature_column.categorical_column_with_vocabulary_list(feature_name, dataframe[feature_name].unique())
        new_feature_as_indicator   = feature_column.indicator_column(new_feature_as_categorical) 
        feature_columns.append(new_feature_as_indicator)
        input_layer[feature_name] = tensorflow.keras.Input(shape=(1,), name = feature_name, dtype = tensorflow.string)


    # add embedded features
    for feature_name in embedded_features:
        naive_embedding_size = int(numpy.round(len(dataframe[feature_name].unique())**(0.25)))
        new_feature_as_categorical = feature_column.categorical_column_with_vocabulary_list(feature_name, dataframe[feature_name].unique())
        new_feature_as_embedding   = feature_column.embedding_column(new_feature_as_categorical, naive_embedding_size)
        feature_columns.append(new_feature_as_embedding)
        input_layer[feature_name] = tensorflow.keras.Input(shape=(1,), name = feature_name, dtype = tensorflow.string)

    return feature_columns, input_layer

    
print('inspect everything')
feature_columns, input_layer = build_feature_columns_and_input_layer()
demo(feature_columns)
print('\nInput layer')
print(input_layer)
# Warnings comes out because conda on macOS can have only tensorflow 2.0.0 an not 2.3.14

inspect everything
[[0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 1. 0. 0.]
 ...
 [0. 0. 1. ... 0. 0. 1.]
 [0. 0. 1. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 0. 1.]]

Input layer
{'PhotoAmt': <tf.Tensor 'PhotoAmt_1:0' shape=(None, 1) dtype=float32>, 'Fee': <tf.Tensor 'Fee_1:0' shape=(None, 1) dtype=float32>, 'Age': <tf.Tensor 'Age_1:0' shape=(None, 1) dtype=float32>, 'Type': <tf.Tensor 'Type_1:0' shape=(None, 1) dtype=string>, 'Color1': <tf.Tensor 'Color1_1:0' shape=(None, 1) dtype=string>, 'Color2': <tf.Tensor 'Color2_1:0' shape=(None, 1) dtype=string>, 'Gender': <tf.Tensor 'Gender_1:0' shape=(None, 1) dtype=string>, 'MaturitySize': <tf.Tensor 'MaturitySize_1:0' shape=(None, 1) dtype=string>, 'FurLength': <tf.Tensor 'FurLength_1:0' shape=(None, 1) dtype=string>, 'Vaccinated': <tf.Tensor 'Vaccinated_1:0' shape=(None, 1) dtype=string>, 'Sterilized': <tf.Tensor 'Sterilized_1:0' shape=(None, 1) dtype=string>, 'Health': <tf.Tensor 'Health_1:0' shape=(None, 1) dtype=string>, 'Breed

### Neural model
Now that feature columns are ready, I train a deep neural network model, albeit a very simple one. Let's start by writing down a funcitons that initialized a model object that I can feed to the hyperparameter tuner. Note that in this function the hyperparameters object does not have specific details on the hyperparameter space. Those are defined within the function initialize_model itself.

In [22]:
def initialize_model(hyperparameters):        
    # specify hyperparameter ranges hyperparameter_object 
    node_units  = hyperparameters.Int('units', min_value = 10, max_value = 50, step = 10)
    dropout_val = hyperparameters.Float('dropout', min_value = 0.05, max_value = 0.25, step = 0.05)
    optimizer   = hyperparameters.Choice('optimizer', ['adam', 'ftrl'])
     
        
    # build neural network structure with functional API
    # initialize input_layer and feature_columns rules
    feature_columns, input_layer = build_feature_columns_and_input_layer()
    # Applu feature_columns rules to data from input_layer
    x = tensorflow.keras.layers.DenseFeatures(feature_columns)(input_layer)
    # add relu and dropout layers
    x = layers.Dense(units = node_units, activation='relu')(x)
    x = layers.Dropout(rate = dropout_val)(x)
    x = layers.Dense(units = node_units, activation='relu')(x)
    x = layers.Dropout(rate = dropout_val)(x)
    model_output = layers.Dense(1, activation = 'sigmoid')(x) 
    
    # initialize model from neural network structure
    model = tensorflow.keras.Model(inputs=[v for v in input_layer.values()], outputs = model_output)
    
    # compile model
    model.compile(optimizer = optimizer,
                  loss = tensorflow.keras.losses.BinaryCrossentropy(from_logits = True),
                  metrics = ['accuracy',
                            tensorflow.keras.metrics.Precision(name='precision'),
                            tensorflow.keras.metrics.Recall(name='recall')])
    
    return model

Let's now use this initializer function to set up an hyperparameter tuner.

In [25]:
tuner = kerastuner.Hyperband(initialize_model,
                             objective = 'val_accuracy', 
                             max_epochs = 10,
                             factor = 3,
                             directory = 'logs',
                             project_name = 'hyperparameter_tuning')


We can now run the hyperparameter tuner to find the best hyperparameter configuration

In [26]:
tuner.search(train_ds, epochs = 20, validation_data = val_ds, verbose = 0)

INFO:tensorflow:Oracle triggered exit


Using the log from the tuner, we can now find the best parameter and train the corresponding model

In [27]:
best_hps = tuner.get_best_hyperparameters(num_trials = 1)[0]
print('best model:')
print(f"""nodes:     {best_hps.get('units')}""")
print(f"""dropout:   {best_hps.get('dropout')}""")
print(f"""optimizer: {best_hps.get('optimizer')}""")

best model:
nodes:     30
dropout:   0.1
optimizer: adam


So we can now train the best model

In [28]:
# Build the model with the optimal hyperparameters and train it on the data
model = tuner.hypermodel.build(best_hps)
model.fit(train_ds, epochs = 10, validation_data = val_ds, verbose = 0)

Using the trained model, we can estimate now the performances of the model. I know that I could play with other hyperparameters in this dataset, such as the number of layers to implement, or the size of the embedding for breeds, or joint variables. Such complex optimization is whereas the model evaluation in the test set is outside the scope of this example, so I will move toward validation on the test set instead.

In [30]:
performances_on_validation = model.evaluate(val_ds)
performances_on_test = model.evaluate(test_ds)

73/73 [==============================] - ETA: 0s - loss: 0.5415 - accuracy: 0.6875 - precision: 0.8889 - recall: 0.666 - ETA: 0s - loss: 0.5602 - accuracy: 0.6818 - precision: 0.8349 - recall: 0.705 - ETA: 0s - loss: 0.5704 - accuracy: 0.6652 - precision: 0.8143 - recall: 0.699 - ETA: 0s - loss: 0.5706 - accuracy: 0.6750 - precision: 0.8081 - recall: 0.721 - ETA: 0s - loss: 0.5650 - accuracy: 0.6820 - precision: 0.8166 - recall: 0.726 - ETA: 0s - loss: 0.5671 - accuracy: 0.6825 - precision: 0.8133 - recall: 0.727 - ETA: 0s - loss: 0.5712 - accuracy: 0.6766 - precision: 0.8093 - recall: 0.720 - ETA: 0s - loss: 0.5697 - accuracy: 0.6759 - precision: 0.8108 - recall: 0.719 - 0s 5ms/step - loss: 0.5711 - accuracy: 0.6750 - precision: 0.8086 - recall: 0.7203


In [31]:
print(f"""Performances on validation:
Accuracy:  {performances_on_validation[1]}
Precision: {performances_on_validation[2]}
Recall:    {performances_on_validation[3]}

Performances on test:
Accuracy:  {performances_on_test[1]}
Precision: {performances_on_test[2]}
Recall:    {performances_on_test[3]}""")

Performances on validation:
Accuracy:  0.6565546989440918
Precision: 0.8036605715751648
Recall:    0.7082111239433289

Performances on test:
Accuracy:  0.6750433444976807
Precision: 0.808625340461731
Recall:    0.720288097858429


The performances on the validation and the test set seems in agreement, so the predictive model seems to generalize quite well. This means that I can deploy it. This requires first to save the model for production

In [32]:
model.save(os.path.join(os.getcwd(), 'animal_adoption_model'))

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: /Users/dabol99/Documents/DS projects/Animal_adoptions/animal_adoption_model/assets


which I will be able to deploy on GCP the day I want to pay for their services. Yay!


In [19]:
temp = pandas.DataFrame(model.predict(test_ds))
temp.describe()

,0
count,2308.000000
mean,0.787529
std,0.364780
min,0.000760
25%,0.769985
50%,0.998309
75%,0.999975
max,1.000000


In [32]:
(11537- dataframe['target'].sum())/11537

0.26696714917222847

In [33]:
sum(temp[0]<0.5)/11537

0.04247204645921817